# Concurrency

 * multiprocessing.Process
 * threading.Thread
 
Usage is similar: 
1. Create a derived class of Process or Thread.
2. In the constructor call constructor of Process or Thread
3. Implement `run()` method. It will be called when your Process/Thread is `start()`ed.

In [4]:
from multiprocessing import Process, Lock,RLock, Semaphore
import time

class Philosopher(Process):
    def __init__(self,i,left,right):
        self.left = left
        self.right = right
        self.id = i
        super().__init__()
    def run(self):
    
        for i in range(0,50):
            if self.id % 2 == 0:
                self.left.acquire()
                self.right.acquire()
            else:
                self.right.acquire()
                self.left.acquire()
            print("phil {} is eating".format(self.id))
            time.sleep(0.2)
            self.left.release()
            self.right.release()
            print("phil {} is thinking".format(self.id))
            time.sleep(0.1)
        print("phil {} is terminating".format(self.id))
    

N = 5

locks = [Lock() for i in range(0,N)]

phils=[]
for i in range(0,N):
    p = Philosopher(i, locks[i], locks[i+1] if i+1 < N else locks[0])
    phils.append(p)
    
for phil in phils:
    phil.start()
    
for phil in phils:
    phil.join()


        
        

phil 3 is eating
phil 0 is eating
phil 3 is thinking
phil 0 is thinking
phil 1 is eatingphil 4 is eating

phil 0 is eatingphil 3 is eatingphil 4 is thinkingphil 1 is thinking



phil 3 is thinkingphil 0 is thinkingphil 4 is eating
phil 2 is eating


phil 0 is eatingphil 4 is thinkingphil 2 is thinkingphil 3 is eating



phil 1 is eatingphil 0 is thinkingphil 3 is thinkingphil 4 is eating



phil 2 is eatingphil 1 is thinkingphil 4 is thinkingphil 0 is eating



phil 0 is thinking
phil 1 is eatingphil 3 is eatingphil 2 is thinking


phil 0 is eatingphil 1 is thinkingphil 3 is thinkingphil 2 is eating



phil 0 is thinkingphil 4 is eatingphil 1 is eatingphil 2 is thinking



phil 3 is eatingphil 4 is thinkingphil 1 is thinkingphil 0 is eating



phil 3 is thinkingphil 2 is eatingphil 4 is eatingphil 0 is thinking



phil 3 is eatingphil 0 is eatingphil 4 is thinkingphil 2 is thinking



phil 0 is thinkingphil 4 is eatingphil 3 is thinkingphil 1 is eating



phil 0 is eatingphil 3 is eati

## multiprocessing

`Process` is the main class. It includes synchronization related classes and data structures:

  `Lock, Semaphore, Condition, Value, Array, Queue`
  
`Process(target=`function`, args=(`arguments`)` will create a new  instance (not process yet)

calling `start()` method of the object will create the process and call the parameter function as the entry function.

In [7]:
from multiprocessing import Process
#from threading import Thread
import time

counter = 10

def hello(name):
    global counter
    for i in range(0,5):
        counter += 1
        time.sleep(0.5)
        print("hello " + name, counter)
            
p = Process(target=hello, args=("world",))

q = Process(target=hello, args=("myself",))

p.start() 
q.start()
# p, q and main process are concurrent here

p.join() # wait for p to complete
q.join() # wait for q to complete
print('counter is :',counter)
# back to single process again

hello world 11
hello myself 11
hello world 12
hello myself 12
hello world 13
hello myself 13
hello world 14
hello myself 14
hello world 15hello myself
 15
counter is : 10


 * Multiprocessing environment executes on a separate process. During process creation current set of global variables are copied in a new python interpreter and after that all work isolated.
 * `multiprocess` classes are multi-process aware. They are shared. A global lock or locks passed as parameters will be on a shared environment.

In [8]:
''' Simple communication among two processes.
    Locks are logical entities, process do not
    have to own the lock to release it'''


# Watch this variable. There are two processes incrementing it, three reporting it
# Each process have its own copy
counter = 10
def ping(name,memut,othmut):
    global counter
    for i in range(0,5):
        # wait until my turn
        memut.acquire()
        print(name,counter)
        time.sleep(1)
        # tell other end it is its turn
        othmut.release()
        counter += 1

imut,omut = Lock(), Lock()

pip = Process(target=ping, args=("ping",imut,omut))
pop = Process(target=ping, args=("pong",omut,imut))

# make sure only one (ping enters first)
omut.acquire()
pip.start()
pop.start()
pip.join()
pop.join()
omut.release()
print("in main process: {}".format(counter))
    

ping 10
pong 10
ping 11
pong 11
ping 12
pong 12
ping 13
pong 13
ping 14
pong 14
in main process: 10


In [11]:
from threading import Thread
import time

counter = 10

def hello(name):
    global counter
    for i in range(0,10):
        counter += 1
        time.sleep(0.5)
        print("hello " + name, counter)
            
p = Thread(target=hello, args=("world",))

q = Thread(target=hello, args=("myself",))

p.start() 
q.start()
# p, q and main process are concurrent here

p.join() # wait for p to complete
q.join() # wait for q to complete
print('counter is :',counter)
# back to single process again

hello worldhello myself 12
 12
hello myself 14
hello world 15
hello myself 16
hello world 17
hello myself 18
hello world 19
hello myself 20
hello world 21
hello myself 22
hello world 23
hello myself 24
hello world 25
hello world 26
hello myself 27
hello world 28
hello myself 29
hello world 30
hello myself 30
counter is : 30


In [10]:
''' Simple communication among two processes.
    Locks are logical entities, process do not
    have to own the lock to release it'''

import threading
# Watch this variable. There are two processes incrementing it, three reporting it
# Each process have its own copy
counter = 10
def ping(name,memut,othmut):
    global counter
    for i in range(0,5):
        # wait until my turn
        memut.acquire()
        print(name,counter)
        time.sleep(1)
        # tell other end it is its turn
        othmut.release()
        counter += 1

imut,omut = threading.Lock(), threading.Lock()

pip = Thread(target=ping, args=("ping",imut,omut))
pop = Thread(target=ping, args=("pong",omut,imut))

# make sure only one (ping enters first)
omut.acquire()
pip.start()
pop.start()
pip.join()
pop.join()
omut.release()
print("in main process: {}".format(counter))
    

ping 10
pong 11
ping 12
pong 13
ping 14
pong 15
ping 16
pong 17
ping 18
pong 19
in main process: 20
